### Create documentation

**PROMPT**  
You are a subject matter expert on credit risk policies, regulatory compliance (e.g., Basel III),   
and internal governance standards. I have a set of documents covering regulations, internal   
guidelines, and reference materials. I also have a Table of Contents (ToC) that outlines how I   
want the final policy to be structured. 

Using only the content retrieved from the provided documents and guided by the ToC, please   
create each policy section. For each section, incorporate references to relevant regulatory   
requirements, define any necessary terminology, and outline best practices for credit risk   
assessment, approval workflows, monitoring, reporting, governance, and compliance. 

Keep the language concise, clear, and aligned with standard industry formats. Structure your   
final output according to the headings in the ToC, ensuring a coherent, well-organized policy   
that meets both regulatory and internal standards.

#### CreditRiskPolicyAgent Overview

- **Purpose**: Automate creation and refinement of a Credit Risk Policy document  
- **Components**:  
  - **DocumentCreator**: Fetches context from Chroma and drafts each section  
  - **DocumentEvaluator**: Reviews each draft, providing feedback for improvement  
  - **FinalReviewer**: Conducts a final holistic assessment of the entire policy  

#### Workflow

1. **Draft**:  
   The creator pulls relevant text from the Chroma database.  
   It then produces an initial section draft.  

2. **Evaluate**:  
   The evaluator checks accuracy, clarity, and compliance.  
   Feedback is generated for each section.  

3. **Refine**:  
   The creator refines the draft, based on evaluator feedback.  

4. **Final Review**:  
   Once all sections are ready, the FinalReviewer inspects the entire policy.  
   It ensures coherence and makes any final improvements.  

#### Key Methods

- **draft_section(section_title)**  
  Creates a preliminary version of a policy section.  

- **evaluate_section(draft_text)**  
  Reviews the text for correctness and suggests enhancements.  

- **refine_with_feedback(draft_text, feedback)**  
  Applies feedback to produce a better draft.  

- **review_document(entire_document)**  
  Performs the last check on the policy as a whole.  

#### Usage

1. **Initialize** the CreditRiskPolicyAgent with your API key and Chroma collection.  
2. **Provide** a table of contents for the policy.  
3. **Call** `build_policy`, which executes draft, evaluate, refine, and final review steps.  
4. **Receive** a fully formed policy document, ready for adoption.

In [ ]:
import openai
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

class DocumentCreator:
    """
    Creates and refines policy sections. 
    Pulls context from Chroma by passing the heading, the overall document 
    description, and the specific section description as the query.
    """
    def __init__(self, llm, retriever):
        self.llm = llm
        self.retriever = retriever

        # Prompt for creating a draft from retrieved context
        self.creation_prompt = PromptTemplate(
            template=(
                "You are drafting the '{section_title}' section of a Credit Risk Policy.\n"
                "Overall Document Description:\n{overall_desc}\n\n"
                "Section-Specific Description:\n{section_desc}\n\n"
                "Below are the relevant texts:\n{context}\n\n"
                "Draft the section by incorporating best practices and references."
            ),
            input_variables=[
                "section_title",
                "overall_desc",
                "section_desc",
                "context"
            ]
        )
        self.creation_chain = LLMChain(llm=self.llm, prompt=self.creation_prompt)

        # Prompt for refining a draft based on evaluator feedback
        self.refine_prompt = PromptTemplate(
            template=(
                "Original draft:\n{draft}\n\n"
                "Reviewer feedback:\n{feedback}\n\n"
                "Refine and improve this draft, incorporating all feedback while maintaining clarity."
            ),
            input_variables=["draft", "feedback"]
        )
        self.refine_chain = LLMChain(llm=self.llm, prompt=self.refine_prompt)

    def retrieve_context(self, overall_desc, section_title, section_desc):
        # Combine everything into a single query for Chroma semantic retrieval
        query = (
            f"Overall Document: {overall_desc}\n"
            f"Section Title: {section_title}\n"
            f"Section Description: {section_desc}"
        )
        docs = self.retriever.get_relevant_documents(query)
        return "\n".join([d.page_content for d in docs])

    def draft_section(self, overall_desc, section_title, section_desc):
        context = self.retrieve_context(overall_desc, section_title, section_desc)
        return self.creation_chain.run({
            "section_title": section_title,
            "overall_desc": overall_desc,
            "section_desc": section_desc,
            "context": context
        })

    def refine_with_feedback(self, draft, feedback):
        return self.refine_chain.run({"draft": draft, "feedback": feedback})

class DocumentEvaluator:
    """
    Evaluates individual sections for correctness, clarity, and compliance.
    Provides feedback for improvement.
    """
    def __init__(self, llm):
        eval_template = (
            "You are an independent reviewer.\n\n"
            "Draft section:\n{draft_section}\n\n"
            "1) Evaluate correctness, clarity, and compliance.\n"
            "2) Propose improvements.\n"
            "3) Provide concise, constructive feedback."
        )
        self.evaluation_prompt = PromptTemplate(
            template=eval_template,
            input_variables=["draft_section"]
        )
        self.chain = LLMChain(llm=llm, prompt=self.evaluation_prompt)

    def evaluate_section(self, draft_section):
        return self.chain.run({"draft_section": draft_section})

class FinalReviewer:
    """
    Conducts a holistic review of the assembled policy document. 
    Ensures coherence, consistency, and completeness.
    """
    def __init__(self, llm):
        final_review_template = (
            "You are a senior reviewer. Below is the entire policy.\n\n"
            "{full_document}\n\n"
            "1) Check consistency and completeness.\n"
            "2) Suggest improvements.\n"
            "3) Provide the final revised text."
        )
        self.final_prompt = PromptTemplate(
            template=final_review_template,
            input_variables=["full_document"]
        )
        self.chain = LLMChain(llm=llm, prompt=self.final_prompt)

    def review_document(self, full_document):
        return self.chain.run({"full_document": full_document})

class CreditRiskPolicyAgent:
    """
    Orchestrates creation, section-level evaluation, refinement, and a final review.
    """
    def __init__(self, api_key, chroma_collection):
        openai.api_key = api_key
        embeddings = OpenAIEmbeddings(openai_api_key=api_key)
        store = Chroma(collection_name=chroma_collection, embedding_function=embeddings)
        llm = OpenAI(model_name="gpt-4o")

        self.creator = DocumentCreator(llm, store.as_retriever())
        self.evaluator = DocumentEvaluator(llm)
        self.final_reviewer = FinalReviewer(llm)

    def build_policy(self, overall_desc, toc_dict):
        """
        toc_dict is a dictionary: {
          "section_1": {
             "title": "Introduction and Scope",
             "description": "High-level overview, disclaimers, and scope"
          },
          "section_2": ...
        }
        """
        refined_sections = {}

        # 1) Draft, evaluate, refine each section
        for key, section_info in toc_dict.items():
            s_title = section_info["title"]
            s_desc = section_info["description"]

            draft = self.creator.draft_section(overall_desc, s_title, s_desc)
            feedback = self.evaluator.evaluate_section(draft)
            refined_draft = self.creator.refine_with_feedback(draft, feedback)

            refined_sections[s_title] = refined_draft

        # 2) Combine sections and run a final review
        full_document = ""
        for s_title in refined_sections:
            full_document += f"{s_title}\n\n{refined_sections[s_title]}\n\n"

        final_result = self.final_reviewer.review_document(full_document)
        return final_result

def main():
    api_key = "YOUR_OPENAI_API_KEY"
    chroma_collection = "my_preembedded_docs"

    # Overall purpose or context of the entire document
    overall_doc_description = (
        "This document outlines the company's Credit Risk Policy in line "
        "with Basel III and internal guidelines."
    )

    # Table of contents: each section has a title + a short description
    toc = {
        "section_1": {
            "title": "Introduction and Scope",
            "description": "High-level overview, disclaimers, and scope"
        },
        "section_2": {
            "title": "Definitions and Terminology",
            "description": "List and define all key terms used in the policy"
        },
        "section_3": {
            "title": "Risk Assessment Methodology",
            "description": "Describe how risks are identified, measured, and scored"
        },
        "section_4": {
            "title": "Credit Approval Workflow",
            "description": "Detail approval steps, roles, and responsibilities"
        },
        "section_5": {
            "title": "Monitoring and Reporting",
            "description": "Explain ongoing monitoring frameworks and reporting lines"
        },
        "section_6": {
            "title": "Governance and Compliance",
            "description": "Outline governance structure and compliance requirements"
        }
    }

    # Build the policy using the orchestrator
    agent = CreditRiskPolicyAgent(api_key, chroma_collection)
    final_document = agent.build_policy(overall_doc_description, toc)

    print("FINAL POLICY DOCUMENT:\n")
    print(final_document)

if __name__ == "__main__":
    main()